In [14]:
from pickle import TRUE
import time
import requests
import bs4
import os


In [20]:
with open('keywords.txt', 'r') as f:
    keywords = f.readlines()

# into list
keywords = [k.strip() for k in keywords]

print(keywords)

['民進黨', '國民黨', '民眾黨', '時代', '中共', '炒房', '垃圾', '低能', '智障', '二戰', '習近平', '蔡英文', '蘇貞昌', '陳水扁', '馬英九', '柯文哲', '高虹安', '蔣萬安', '吳怡農', '賴清德', '苗博雅', '館長', '拜登', '川普', '唐鳳', '歐巴馬', '中國', '大陸', '中華', '民國', '美國', '台灣', '日本', '香港', '韓國', '法國', '德國', '英國', '國家', '西方', '總統', '大選', '選舉', '學店', '肥宅', '鄉民', '民主', '共產', '笑話', '中山', '中正', '網軍', '崩潰', '褲子', '支那', '台北', '新北', '新竹', '高雄', '台南', '台中', '桃園', '中壢', '中立', '民代', '議員', '立委', '助理', '民調', '網友', '世足', '新冠', '武漢', '肺炎', '流感', '確診', '國會', '立院', '立法', '內閣', '側翼', '貪污', '抄襲', '詐騙', '抖音', '法官', '八卦', '爆卦', '問卦', '佳德', '臉書', '藍營', '綠營', '獨立', '白宮', '警察', '勞工']


In [21]:
# 設定Cookie
cookies = {
    'over18': '1'
}

In [26]:
def getCurrentPageLink(URL):
    # 發送get 請求 到 ptt 八卦版
    response = requests.get(URL, cookies=cookies)
    time.sleep(1)
    #  把網頁程式碼(HTML) 丟入 bs4模組分析
    soup = bs4.BeautifulSoup(response.text, "html.parser")

    ## 查找html 元素 抓出內容
    main_container = soup.find(id='main-container')

    divs = main_container.find_all('div', 'title')

    a_Set = []
    links = []

    for div in divs:
        a = div.find_all('a')
        # [<a href="/bbs/Gossiping/M.1648453620.A.ED8.html">[協尋] 3/25晚上9：30左右 國一北上往苗栗頭份路</a>]
        # print(a)
        # print(type(a))  # <class 'bs4.element.ResultSet'>
        a_Set.extend(a)


    # 取得當前頁數文章連結
    for a in a_Set:
        link = "https://www.ptt.cc/" + a.get('href')
        links.append(link)  
        print(link)
    return links


def KeywordSearch(key):
    currentPage = 1
    all_links = []
    while(True):
        # 連結
        Gossiping_URL = "https://www.ptt.cc/bbs/Gossiping/search?page="
        BASE_URL = Gossiping_URL + str(currentPage) + "&q="
        # print(BASE_URL)
        URL = BASE_URL + key
        #print(URL)
        try:
            links = getCurrentPageLink(URL)
            time.sleep(2)
            all_links.extend(links)
            print('current page is {}'.format(currentPage))
            if currentPage > 15:
                break
            else:
                currentPage += 1
        except AttributeError:
            break
    return all_links


def getCommentsFromEveryLink(links):
    all_messages = []
    print('there are {} links'.format(len(links)))
    index = 1
        
    for link in links:
        if index > 300:
            print('Over 300 links, break') # max:1000 = 20 links * 50 pages
            break
        response = requests.get(link, cookies=cookies)
        time.sleep(2)
        link_messages = []
        #將原始碼做整理
        soup = bs4.BeautifulSoup(response.text, "html.parser")
        #使用find_all()找尋特定目標
        # 抓出所有留言
        comments = soup.find_all("div", class_="push")
        # 假設 user_ids 和 messages 是包含相同數量元素的兩個列表
        # 定義一個空字典，用來存放每個使用者ID的留言內容
        user_messages = {}
        try:
            for comment in comments:
                user_id = comment.find(
                    "span", class_="push-userid").getText().replace(':', '').strip()
                message = comment.find(
                    "span", class_="push-content").getText().replace(':', '').strip()
                # 如果該使用者ID尚未在字典中出現，則將其加入字典中
                if user_id not in user_messages:
                    user_messages[user_id] = []
                # 將留言內容加入該使用者ID的留言內容列表中
                user_messages[user_id].append(message)
        except:
            continue    
        link_messages.append(user_messages)
        print('link {} is done.'.format(index))
        index += 1
        all_messages.extend(link_messages)
    return all_messages

In [29]:
dir_name = './output/'

file_exists_names = [f for f in os.listdir(dir_name) if not f.startswith('.')]
print(file_exists_names)

for keyword in keywords:
    
    if keyword+'.txt' not in file_exists_names:
        
        my_links = KeywordSearch(keyword)
        all_messages = getCommentsFromEveryLink(my_links)
        print(keyword)
    
    
        with open(dir_name + keyword +'.txt', 'w') as f:
            # 遍歷list中的每個字典
            for d in all_messages:
                # 將字典中的每個list的每個element轉換成字串
                for user_id, message in d.items():
                    d[user_id] = "".join(message)
        
            # 寫入每個字典
            for d in all_messages:
                # 將字典中的每個字串寫入文件
                for message in d.values():
                    f.write(message + "\n")

['民進黨.txt']
https://www.ptt.cc//bbs/Gossiping/M.1671159277.A.D4A.html
https://www.ptt.cc//bbs/Gossiping/M.1671157190.A.17B.html
https://www.ptt.cc//bbs/Gossiping/M.1671155678.A.971.html
https://www.ptt.cc//bbs/Gossiping/M.1671150581.A.FF1.html
https://www.ptt.cc//bbs/Gossiping/M.1671120354.A.4E8.html
https://www.ptt.cc//bbs/Gossiping/M.1671113966.A.813.html
https://www.ptt.cc//bbs/Gossiping/M.1671100640.A.B0B.html
https://www.ptt.cc//bbs/Gossiping/M.1671095832.A.43D.html
https://www.ptt.cc//bbs/Gossiping/M.1671079684.A.854.html
https://www.ptt.cc//bbs/Gossiping/M.1671075964.A.ADD.html
https://www.ptt.cc//bbs/Gossiping/M.1671069868.A.D74.html
https://www.ptt.cc//bbs/Gossiping/M.1671059713.A.1B1.html
https://www.ptt.cc//bbs/Gossiping/M.1671036344.A.81E.html
https://www.ptt.cc//bbs/Gossiping/M.1671035213.A.E52.html
https://www.ptt.cc//bbs/Gossiping/M.1671031113.A.E9D.html
https://www.ptt.cc//bbs/Gossiping/M.1671030709.A.CD6.html
https://www.ptt.cc//bbs/Gossiping/M.1671028507.A.A6B.html
ht

current page is 7
https://www.ptt.cc//bbs/Gossiping/M.1670220540.A.101.html
https://www.ptt.cc//bbs/Gossiping/M.1670220433.A.4CA.html
https://www.ptt.cc//bbs/Gossiping/M.1670220366.A.AE7.html
https://www.ptt.cc//bbs/Gossiping/M.1670220179.A.1DA.html
https://www.ptt.cc//bbs/Gossiping/M.1670219960.A.705.html
https://www.ptt.cc//bbs/Gossiping/M.1670219201.A.271.html
https://www.ptt.cc//bbs/Gossiping/M.1670218931.A.210.html
https://www.ptt.cc//bbs/Gossiping/M.1670218792.A.C6F.html
https://www.ptt.cc//bbs/Gossiping/M.1670218641.A.E88.html
https://www.ptt.cc//bbs/Gossiping/M.1670218596.A.CE2.html
https://www.ptt.cc//bbs/Gossiping/M.1670218575.A.B14.html
https://www.ptt.cc//bbs/Gossiping/M.1670218516.A.EB5.html
https://www.ptt.cc//bbs/Gossiping/M.1670218204.A.D9B.html
https://www.ptt.cc//bbs/Gossiping/M.1670218027.A.242.html
https://www.ptt.cc//bbs/Gossiping/M.1670217548.A.1CB.html
https://www.ptt.cc//bbs/Gossiping/M.1670217023.A.A12.html
https://www.ptt.cc//bbs/Gossiping/M.1670216632.A.3C9.h

current page is 14
https://www.ptt.cc//bbs/Gossiping/M.1669788734.A.587.html
https://www.ptt.cc//bbs/Gossiping/M.1669788217.A.F8C.html
https://www.ptt.cc//bbs/Gossiping/M.1669788114.A.759.html
https://www.ptt.cc//bbs/Gossiping/M.1669787772.A.4A9.html
https://www.ptt.cc//bbs/Gossiping/M.1669787297.A.864.html
https://www.ptt.cc//bbs/Gossiping/M.1669786467.A.BB1.html
https://www.ptt.cc//bbs/Gossiping/M.1669785964.A.440.html
https://www.ptt.cc//bbs/Gossiping/M.1669785301.A.322.html
https://www.ptt.cc//bbs/Gossiping/M.1669785153.A.9B3.html
https://www.ptt.cc//bbs/Gossiping/M.1669780972.A.FE3.html
https://www.ptt.cc//bbs/Gossiping/M.1669776200.A.C32.html
https://www.ptt.cc//bbs/Gossiping/M.1669774523.A.D44.html
https://www.ptt.cc//bbs/Gossiping/M.1669773629.A.9CA.html
https://www.ptt.cc//bbs/Gossiping/M.1669773315.A.32C.html
https://www.ptt.cc//bbs/Gossiping/M.1669772076.A.99F.html
https://www.ptt.cc//bbs/Gossiping/M.1669771858.A.9CE.html
https://www.ptt.cc//bbs/Gossiping/M.1669770702.A.EC9.

current page is 21
https://www.ptt.cc//bbs/Gossiping/M.1669564090.A.477.html
https://www.ptt.cc//bbs/Gossiping/M.1669563569.A.955.html
https://www.ptt.cc//bbs/Gossiping/M.1669562226.A.BDE.html
https://www.ptt.cc//bbs/Gossiping/M.1669559696.A.1AF.html
https://www.ptt.cc//bbs/Gossiping/M.1669558944.A.43D.html
https://www.ptt.cc//bbs/Gossiping/M.1669558439.A.292.html
https://www.ptt.cc//bbs/Gossiping/M.1669557304.A.B30.html
https://www.ptt.cc//bbs/Gossiping/M.1669555368.A.A72.html
https://www.ptt.cc//bbs/Gossiping/M.1669555116.A.0FA.html
https://www.ptt.cc//bbs/Gossiping/M.1669553381.A.3A9.html
https://www.ptt.cc//bbs/Gossiping/M.1669552926.A.62C.html
https://www.ptt.cc//bbs/Gossiping/M.1669550752.A.D82.html
https://www.ptt.cc//bbs/Gossiping/M.1669547761.A.22B.html
https://www.ptt.cc//bbs/Gossiping/M.1669546889.A.7B9.html
https://www.ptt.cc//bbs/Gossiping/M.1669546535.A.361.html
https://www.ptt.cc//bbs/Gossiping/M.1669543776.A.256.html
https://www.ptt.cc//bbs/Gossiping/M.1669541147.A.159.

current page is 28
https://www.ptt.cc//bbs/Gossiping/M.1669467329.A.F36.html
https://www.ptt.cc//bbs/Gossiping/M.1669467099.A.8DD.html
https://www.ptt.cc//bbs/Gossiping/M.1669467090.A.F82.html
https://www.ptt.cc//bbs/Gossiping/M.1669467049.A.823.html
https://www.ptt.cc//bbs/Gossiping/M.1669466900.A.C15.html
https://www.ptt.cc//bbs/Gossiping/M.1669466820.A.AC3.html
https://www.ptt.cc//bbs/Gossiping/M.1669466801.A.AFD.html
https://www.ptt.cc//bbs/Gossiping/M.1669466562.A.F8D.html
https://www.ptt.cc//bbs/Gossiping/M.1669466303.A.8B2.html
https://www.ptt.cc//bbs/Gossiping/M.1669466000.A.CC8.html
https://www.ptt.cc//bbs/Gossiping/M.1669465620.A.3B7.html
https://www.ptt.cc//bbs/Gossiping/M.1669465442.A.163.html
https://www.ptt.cc//bbs/Gossiping/M.1669465222.A.656.html
https://www.ptt.cc//bbs/Gossiping/M.1669465184.A.0CD.html
https://www.ptt.cc//bbs/Gossiping/M.1669465037.A.9FA.html
https://www.ptt.cc//bbs/Gossiping/M.1669464727.A.C63.html
https://www.ptt.cc//bbs/Gossiping/M.1669464473.A.B60.

...

current page is 26
https://www.ptt.cc//bbs/Gossiping/M.1668085734.A.D12.html
https://www.ptt.cc//bbs/Gossiping/M.1668085145.A.E97.html
https://www.ptt.cc//bbs/Gossiping/M.1668084929.A.05C.html
https://www.ptt.cc//bbs/Gossiping/M.1668077717.A.CE6.html
https://www.ptt.cc//bbs/Gossiping/M.1668077002.A.7E8.html
https://www.ptt.cc//bbs/Gossiping/M.1668076802.A.76B.html
https://www.ptt.cc//bbs/Gossiping/M.1668076598.A.AFA.html
https://www.ptt.cc//bbs/Gossiping/M.1668076393.A.181.html
https://www.ptt.cc//bbs/Gossiping/M.1668075174.A.2AF.html
https://www.ptt.cc//bbs/Gossiping/M.1668074689.A.B13.html
https://www.ptt.cc//bbs/Gossiping/M.1668074204.A.C10.html
https://www.ptt.cc//bbs/Gossiping/M.1668074094.A.DF8.html
https://www.ptt.cc//bbs/Gossiping/M.1668072625.A.735.html
https://www.ptt.cc//bbs/Gossiping/M.1668070444.A.C2E.html
https://www.ptt.cc//bbs/Gossiping/M.1668068274.A.A90.html
https://www.ptt.cc//bbs/Gossiping/M.1668067531.A.5A5.html
https://www.ptt.cc//bbs/Gossiping/M.1668066815.A.0E9.

current page is 33
https://www.ptt.cc//bbs/Gossiping/M.1665323155.A.099.html
https://www.ptt.cc//bbs/Gossiping/M.1665318339.A.D4A.html
https://www.ptt.cc//bbs/Gossiping/M.1665316958.A.C21.html
https://www.ptt.cc//bbs/Gossiping/M.1665248115.A.CD0.html
https://www.ptt.cc//bbs/Gossiping/M.1665209784.A.7B5.html
https://www.ptt.cc//bbs/Gossiping/M.1665207398.A.742.html
https://www.ptt.cc//bbs/Gossiping/M.1665201123.A.E11.html
https://www.ptt.cc//bbs/Gossiping/M.1665195870.A.DB6.html
https://www.ptt.cc//bbs/Gossiping/M.1665191292.A.D5B.html
https://www.ptt.cc//bbs/Gossiping/M.1665181313.A.6DE.html
https://www.ptt.cc//bbs/Gossiping/M.1665103641.A.8D6.html
https://www.ptt.cc//bbs/Gossiping/M.1665027669.A.3E8.html
https://www.ptt.cc//bbs/Gossiping/M.1664984166.A.129.html
https://www.ptt.cc//bbs/Gossiping/M.1664891230.A.6A2.html
https://www.ptt.cc//bbs/Gossiping/M.1664764627.A.B34.html
https://www.ptt.cc//bbs/Gossiping/M.1664725188.A.47D.html
https://www.ptt.cc//bbs/Gossiping/M.1664721638.A.00D.

current page is 40
https://www.ptt.cc//bbs/Gossiping/M.1661232225.A.E93.html
https://www.ptt.cc//bbs/Gossiping/M.1661219582.A.9E4.html
https://www.ptt.cc//bbs/Gossiping/M.1661187059.A.32F.html
https://www.ptt.cc//bbs/Gossiping/M.1661171400.A.5B0.html
https://www.ptt.cc//bbs/Gossiping/M.1661170532.A.0EF.html
https://www.ptt.cc//bbs/Gossiping/M.1661164250.A.E5B.html
https://www.ptt.cc//bbs/Gossiping/M.1661162385.A.1B9.html
https://www.ptt.cc//bbs/Gossiping/M.1661160049.A.60E.html
https://www.ptt.cc//bbs/Gossiping/M.1661154790.A.DDB.html
https://www.ptt.cc//bbs/Gossiping/M.1661150938.A.F5A.html
https://www.ptt.cc//bbs/Gossiping/M.1660985342.A.529.html
https://www.ptt.cc//bbs/Gossiping/M.1660983650.A.018.html
https://www.ptt.cc//bbs/Gossiping/M.1660980867.A.CAE.html
https://www.ptt.cc//bbs/Gossiping/M.1660707412.A.494.html
https://www.ptt.cc//bbs/Gossiping/M.1660686096.A.114.html
https://www.ptt.cc//bbs/Gossiping/M.1660657254.A.094.html
https://www.ptt.cc//bbs/Gossiping/M.1660616817.A.5FA.

current page is 47
https://www.ptt.cc//bbs/Gossiping/M.1657499468.A.FB0.html
https://www.ptt.cc//bbs/Gossiping/M.1657498211.A.363.html
https://www.ptt.cc//bbs/Gossiping/M.1657471468.A.E96.html
https://www.ptt.cc//bbs/Gossiping/M.1657461180.A.414.html
https://www.ptt.cc//bbs/Gossiping/M.1657450082.A.746.html
https://www.ptt.cc//bbs/Gossiping/M.1657438573.A.9FE.html
https://www.ptt.cc//bbs/Gossiping/M.1657436474.A.B0F.html
https://www.ptt.cc//bbs/Gossiping/M.1657419750.A.D0F.html
https://www.ptt.cc//bbs/Gossiping/M.1657375738.A.F58.html
https://www.ptt.cc//bbs/Gossiping/M.1657354153.A.DA7.html
https://www.ptt.cc//bbs/Gossiping/M.1657332389.A.76D.html
https://www.ptt.cc//bbs/Gossiping/M.1657293305.A.F81.html
https://www.ptt.cc//bbs/Gossiping/M.1657278858.A.F99.html
https://www.ptt.cc//bbs/Gossiping/M.1657275117.A.C8B.html
https://www.ptt.cc//bbs/Gossiping/M.1657270999.A.AE6.html
https://www.ptt.cc//bbs/Gossiping/M.1657270689.A.93E.html
https://www.ptt.cc//bbs/Gossiping/M.1657270441.A.52A.

ConnectionError: HTTPSConnectionPool(host='www.ptt.cc', port=443): Max retries exceeded with url: //bbs/Gossiping/M.1669983717.A.1DD.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10b9892e0>: Failed to establish a new connection: [Errno 51] Network is unreachable'))

## 單個關鍵字測試

In [57]:
keyword = str(input("Enter the keyword to search:"))
#User intput:
file_name = str(
    input("Enter the name you want for the new output file (Maybe the keyword):"))
my_links = KeywordSearch(keyword)

all_messages = getCommentsFromEveryLink(my_links)
#print(all_messages)

#寫入檔案中
with open(file_name +'.txt', 'w') as f:
    # 遍歷list中的每個字典
    for d in all_messages:
        # 將字典中的每個list的每個element轉換成字串
        for user_id, message in d.items():
            d[user_id] = "".join(message)
    
    #print(all_messages)
    
    # 寫入每個字典
    for d in all_messages:
        # 將字典中的每個字串寫入文件
        for message in d.values():
            f.write(message + "\n")


Enter the keyword to search:低能兒
Enter the name you want for the new output file (Maybe the keyword):低能兒
https://www.ptt.cc//bbs/Gossiping/M.1669441498.A.DD1.html
https://www.ptt.cc//bbs/Gossiping/M.1667604157.A.7A6.html
https://www.ptt.cc//bbs/Gossiping/M.1665650668.A.DAF.html
https://www.ptt.cc//bbs/Gossiping/M.1664086386.A.20A.html
https://www.ptt.cc//bbs/Gossiping/M.1664079504.A.D85.html
https://www.ptt.cc//bbs/Gossiping/M.1664069436.A.75B.html
https://www.ptt.cc//bbs/Gossiping/M.1664068462.A.445.html
https://www.ptt.cc//bbs/Gossiping/M.1663217819.A.8B9.html
https://www.ptt.cc//bbs/Gossiping/M.1662868350.A.CF1.html
https://www.ptt.cc//bbs/Gossiping/M.1662778614.A.27C.html
https://www.ptt.cc//bbs/Gossiping/M.1661758898.A.549.html
https://www.ptt.cc//bbs/Gossiping/M.1661177370.A.4B4.html
https://www.ptt.cc//bbs/Gossiping/M.1661175335.A.D37.html
https://www.ptt.cc//bbs/Gossiping/M.1660812432.A.691.html
https://www.ptt.cc//bbs/Gossiping/M.1659996908.A.1E1.html
https://www.ptt.cc//bbs/Go

kent 個人覺得比蓮蓉好吃
peasuka 比棗泥好吃
owhaha 這種文章是什麼低能兒在發的文的....
doro0202 蓮蓉真的很問號
JFNfrog 超愛奶酥
koromo1991 綠色不應該是鼻屎口味嗎
AGIknight 你吃的是不是去年的月餅
Riesz 你應該吃到的是綠豆碰  在中立混亂陣營它是
Riesz 一種廣義的月餅
mitshy 那你吃椰酥
wonder007 奶酥真爽
godbar 棗泥才是好嗎 噁心死了
asyousaid 棗泥最噁
Dragonz 棗泥很好吃啊，這麼多人討厭啊
airgp32002 那這篇低能兒的文章又是誰發的
link 8 is done.
AGwet Dinner...
k385476916 好了啦 兩個一起下去
osalucard 是死豬不怕滾水燙 票少不怕更少
chuchichan 哈們
milk7054 本來就民進黨的棄子，用來吸仇恨值保住
saygogo 影片不就你們自己拍的那有造假,在廁所上
milk7054 南部幾顆西瓜
saygogo 面往下探出頭看隔壁上廁所的不是陳時中
saygogo ？
lionghe 這次我支持...
jeff50363 哈門
jma306 陳時中  是團隊要告  我也不清楚啦
only3v 哈們
spzper 這些不就之前你挺出來的~是忘記你挺誰
airaqua 哈們
pokyhoyo 阿扣這麼直白肯定告得成 又要幫時鐘加年
pokyhoyo 終了
huckebein12 不要以為跟風罵克拉克你刺青仔就能
huckebein12 洗白了，兩個抱一起消失去吧
louis82828 哈們
ilovecat5566 這也能被你發現
toshizo 這篇不要操作推文
nonocom 低能兒在哪裡??
nonocom 哦哦 此低非彼低 我覺得亂嗆要殺人比較低
ChenYenChou 智商比呼麻的低 被稱為dinner兒剛好
GOD5566 吸毒的都看不下去  時中該檢討了==
borissun 中共同路人算不算毀謗
databbs 真正低能是誰 非常明顯了
Chia2323 說個笑話 民進黨說別人造假
loveoichi 連呼麻仔都可以嗆爛酒空仔，可憐吶
suhaw 陳時中跟姚文智作用一樣，就是要打第三勢力
hdotistyle 被哈們嗆爆 可憐酒空
suhaw 會不上是一定，只是不

domon0525 這部還蠻經典的
viable https//youtu.be/8CD06hC1KGU
viable 外邦都來學漢字啦好有面子
viable 崇洋都被醫治～～
link 16 is done.
kent 有臭耳
chinnez https//i.imgur.com/VhI9GCL.jpg
luckyalbert 你要不要先吃五樓一拳？
FZ5566 年過三十中年老漢聽力退化
luckyalbert 蓋
citizen5566 蓋
luckyalbert 讚，先吃我一拳吧
luckyalbert 4樓6樓同一位幫灌一拳感恩
citizen5566 灌
ev331 好可怕啊 要限制言論yy
kkchen 我每次講話講了三次，對方都聽不清楚，真
kkchen 的懷疑對方耳朵是不是壞了，害我每次講話
kkchen 都得大聲說。
snow3804 沒人跟他說這樣不行，包括你
zzff92 講得好像你敢打一樣 如果是有人敢糾正我
zzff92 大聲 我一定讓他後悔出生過
pttenjk 宅？我家隔壁的國中牲和外勞常如野狗在
pttenjk 咆哮 原來是宅的原因 來台會變宅真奇怪
blueskyqoo 貓 狗 有在控制音量膩
markwuuu 樓上正解
markwuuu 媽的昨天在河堤跑步兩個歐巴桑散步講話
markwuuu 也超大聲，真想一腳踹下去河裡面
a8239060 沒法律你可能也被其他看你不爽的人揍了
remarkand 沒法律 你可能會後悔出生在世界上
YJM1106 葉問：
coco543961 我說話大聲不代表我沒禮貌
weselyong 我很常聽到這種人是講某種語言…超煩
link 17 is done.
love0504 ？
qt359101 講中文
JamesForrest who?
viable Elon Musk
viable 0.0
link 18 is done.
spzper 難怪清很多
lupin2401 七個月
koromo1991 你可以進去陪陪他啊
Stupidog5566 英雄惜英雄 ㄏ
kcclasaki 樓下也是低能兒 樓樓下也是
calvin00300 http//i.imgur.com/X3MJPC0.jpg
monitor 少一個護航仔而已
ev331 蛤 他也會被桶
toomtoom 各位放心 我沒事 嘻嘻


highmangogo 邏輯不通
iPolo3 小時候？
SHINJOY 你爸那年代沒買房很可惜 算輸了
weiyudanbing 幾十年前拿出來講？結果你南漂了
lupin2401 你長得很年輕 你可以租在學區阿
lupin2401 我在文化旁新安居村租過一年
lupin2401 沒離現在很久誒 那邊有個上班族租在那
lupin2401 我們不是北漂喔 只是上一代生太多
hong414 父母那年代沒買房真的輸爆 還欠錢omg~
hong414 欠就算 老爸還隱瞞shit
lupin2401 沒欠錢誒 我爸倒是戒菸了
justeat 你爸的年代不一樣啊
justeat 公務員的薪資比通膨少很多耶
PRME 社子......
link 30 is done.
gn01693664 苦無蹭蹭俠
yoyodiy 掰  政問
a949949 蓋
atelier 他掌握財富密碼在那邊耍猴戲給你們看
atelier 繼續給他舞台的人才是低能兒
bowen5566 哈哈哈
love12548 綠青蛙真的噁心
foredoom 偶們不知道
dk611216 受眾開心就好 關你屁事
rei196 中輟生你覺得呢
dotzai 不然勒 還能4三洨？
maximu 苦無
pf270801 我以為你要說天天
veelah1225 政問？
andy279279 逢綠比必順遇柯必反的雙標仔
tzu0217 4
rd5186 他就為了黨放棄自我良知跟思考的其中一個
GLung 第一篇還在那邊假裝認同討厭新名詞反酸柯
GLung 結果被打爆臉
ronga 獲得財富密碼怎麼會是低能兒呢
yao03911 有人可以統計一下苦無大師被打臉幾次了
samhou6 噁心智障雙標低能塔綠班
sai2100sai 就演給想看的人看
sbthans 市議員監督市政是ok的，不過監督到臉被打
sbthans 爆就很好笑了
pippen2002 https//i.imgur.com/a4hgT98.jpg超?
benting 苦無證據
leeoon 不要侮辱低能兒
bang71013 高潮了
zhalao 4，淪落成這樣
headiron 4
link 31 is done.
TatsuyaShiba 三樓那個內射七歲女童的變態吧
nopennogame 這時間也太沒水準
ridecule 哈哈是我啦
leon771

gay7788 唯一信癢
xox5678 低能綠共
ted01234567 誰？
s970086 小護士
ggbi4zzz 樓下維尼
spzper 可裡面沒有我阿XDD
bill403777 統神也說過會看八卦版新聞獲取資訊==
spzper 看到在八卦發文以為再自介www
gankgf 我要支持g社
Roderickey 我都看八卦買股票
Doublecute 統神聊天室白癡才多==
HayamaAkito 我都讓他們先上車 嘻嘻
bill403777 充分揭露統神對八卦的看法 有說錯嗎？
zold 五十步笑百步 ............
wmud 知道了
f051372 這胖子也是事後諸葛阿 你以為他多高明
f051372 還很會賴皮
ctw01 鄉民本來就很好騙
uhbygv45 我沒被騙欸 哪邊好操控了
Heisme 呵呵 你各位被低能兒嘲笑低能
EricMartin 鼓吹統一的諧咖 還唯一信癢咧笑死
link 51 is done.
Silent 一樓那種
claude3 有可能傷害別人的低能啊
Roooz 五樓那個肛交狂魔
michaelwu 你要戴可以自己戴啊 不用自介
liuyc888 低能兒才會戴口罩 寧願染疫情
salix 我建議你也不要出門了，病原體會附
salix 著在你身上帶
ultratimes 開車有載人也要戴口罩就是爛啊
agong 有金身
salix 回家很危險
horseorange 覺得跟路人吃飯不用擋但外出要戴的
QQBB 反串你要先講啊
horseorange 是哪個d能
michaelwu 你自殺也可以對世界更有幫助喔 去啊
Leika 苦命人很多 他們先前配合戴並不是愛惜性命
ufowilliam 口罩裡面放屎邊舔都沒人知道 嘻嘻
derrick1220 1F…
Leika 只是為了苟活的消費而已
evan000000 怎麼辦 我覺得你在反串
hakuoro 你爸一定沒戴保險套才會搞出你這種人吧
tpck 自己抵抗力差才會確診 別人戴不戴關你屁事
Leika 鄭捷：「我想死只是不想(敢)自己來…」很
Leika 多人其實不怕死…
ooigi 不就本板的人嗎
ooigi 除了我 哈哈
liuyc888 路上戴口罩的低能真的不少
Leika 若病死了 也解脫了很多壓力
stvn2567 鐘粉專家：健康不用戴 自癒不用篩
uk

setsunabs 不會
rb731024 思覺失調？
brycejack 不會
setsunabs 精障如未滿7不罰
derrick1220 請問我們長的像D能兒嗎？
MEGAMewtwoX 我看起來像法官嗎
brycejack 台灣不是還打算推免死嗎
nekoares 你的低能是指智能障礙嗎?
malindorothy 樓下低能兒
hyperdunk 台大宅王殺人都沒死刑了
yorkyoung 我是低能兒
malindorothy 樓上低能兒
slovea 都低能要怎麼殺人
setsunabs 遇正妹強姦她自稱智障pass
GGrunrundela 在鬼島 因為防衛或是受不了父母虐
GGrunrundela 待反殺這種的才會背叛死刑
smpian 口供啦
smpian 不會選字比低能兒還可悲
save 你剛看完台灣啟示錄吼
super10000 不太可能
link 64 is done.
iPadProPlus 五樓也是
adamlovedogc 一樓才是低能兒
peterch 推文小心
yorkyoung 推文小心
adamlovedogc 改五樓
SRNOB 多看rd英文
t00012 驚險蓋
pengjoker 趕快上104啦 還在這發廢文
february2 原PO做了什麼事？
jacket664 好
theeht ID錯誤
shimazu 可憐吶
sexygnome 你這冒牌貨
GGrunrundela 恩恩
fraternity 推文小心 被釣魚啦
j55888819 低先生你好
paris0826 樓下喜憨兒
C13H16ClNO 看來原po不敢講做了什麼應該有難言之
marktak 洨
zakijudelo D哥好
freakclaw 喔 低能兒
crimson11 唉滴錯誤
dtwilson 推給同事啊 那麼老實幹嘛
Kidmo 沒事兒
link 65 is done.
